In [1]:
import pandas as pd
import numpy as np
import dask.distributed

import tsfresh
import tsfresh.utilities.distribution

#### Load raw data from `processed_raw_data.parquet`

This data was generated in `002_process-raw-data.ipynb`.

In [22]:
df = pd.read_parquet('processed_raw_data.parquet')

# Put `t_ms_rounded` and `segment_id` back in dataframe
df = df.reset_index(['t_ms_rounded', 'segment_id'])

# Drop any rows with NaNs (only about 3%)
df.dropna(how='any', inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3216364 entries, (1600, 'A', 'walking', 1) to (1650, 'S', 'folding', 12)
Data columns (total 14 columns):
 #   Column         Dtype  
---  ------         -----  
 0   t_ms_rounded   int64  
 1   segment_id     int64  
 2   accel_phone_x  float64
 3   accel_phone_y  float64
 4   accel_phone_z  float64
 5   accel_watch_x  float64
 6   accel_watch_y  float64
 7   accel_watch_z  float64
 8   gyro_phone_x   float64
 9   gyro_phone_y   float64
 10  gyro_phone_z   float64
 11  gyro_watch_x   float64
 12  gyro_watch_y   float64
 13  gyro_watch_z   float64
dtypes: float64(12), int64(2)
memory usage: 355.8+ MB


In [23]:
df.head(9)

t_ms_rounded  \
subject_id activity_id activity_name segment_sequence_number                 
1600       A           walking       1                                   0   
                                     1                                  50   
                                     1                                 100   
                                     1                                 150   
                                     1                                 200   
                                     1                                 250   
                                     1                                 300   
                                     1                                 350   
                                     1                                 400   

                                                              segment_id  \
subject_id activity_id activity_name segment_sequence_number               
1600       A           walking       1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   
                                     1                                 1   

                                                              accel_phone_x  \
subject_id activity_id activity_name segment_sequence_number                  
1600       A           walking       1                            -0.364761   
                                     1                            -0.879730   
                                     1                             2.001495   
                                     1                             0.450623   
                                     1                            -2.164352   
                                     1                            -4.332779   
                                     1                            -0.319443   
                                     1                             1.566452   
                                     1                            -0.323746   

                                                              accel_phone_y  \
subject_id activity_id activity_name segment_sequence_number                  
1600       A           walking       1                             8.793503   
                                     1                             9.768784   
                                     1                            11.109070   
                                     1                            12.651642   
                                     1                            13.928436   
                                     1                            13.361191   
                                     1                            13.318359   
                                     1                             9.515274   
                                     1                             5.262665   

                                                              accel_phone_z  \
subject_id activity_id activity_name segment_sequence_number                  
1600       A           walking       1                             1.055084   
                                     1                             1.016998   
                                     1                             2.619156   
                                     1                             0.184555   
                                     1                            -4.422485   
                                     1                            -0.7188

#### Start up Dask Client

In [5]:
client = dask.distributed.Client()

client

Client Scheduler: tcp://127.0.0.1:64184 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.00 GiB


#### Use `tsfresh` to calculate features

In [25]:
sample_df = df[df['segment_id'].isin(df['segment_id'].sample(10, random_state=42))].copy()

y = pd.DataFrame({
    'segment_id': sample_df['segment_id'].values,
    'activity_name': sample_df['segment_id'].index.get_level_values(2),
}).drop_duplicates().set_index('segment_id')['activity_name']


features_df = tsfresh.extract_features(
    timeseries_container = sample_df,
    # y = y,
    column_id = 'segment_id',
    column_sort = 't_ms_rounded',
    distributor = tsfresh.utilities.distribution.ClusterDaskDistributor(
        address = client.scheduler_info()['address']
    )
)

features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 261 to 10422
Columns: 9444 entries, accel_phone_x__variance_larger_than_standard_deviation to gyro_watch_z__matrix_profile__feature_"75"__threshold_0.98
dtypes: float64(9444)
memory usage: 737.9 KB


#### Write `features_df` to disk

In [29]:
features_df.to_parquet('tsfresh_features.parquet', index=True)

#### Shutdown Dask Client

In [30]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
